In [100]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.optimizers import Adam

In [101]:
train_dir = r"Dataset\Train"
validation_dir = r"Dataset\Validation"
test_dir = r"Dataset\Test"

In [102]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [103]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 140002 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.


In [104]:
X_train, y_train = next(train_generator)
X_test, y_test = next(test_generator)
X_val, y_val = next(validation_generator)

Training SVM model

In [105]:
results = {}

In [106]:
svc = SVC()
print("Training SVM model.......")
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)
svc.fit(X_train_reshaped, y_train)
y_pred = svc.predict(X_test_reshaped)
classification_rep = classification_report(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
results["SVM"] = classification_rep, confusion_mat

Training SVM model.......


d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [107]:
rfr = RandomForestClassifier(n_estimators=100)
print("Training Random Forest model.......")
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)
rfr.fit(X_train_flattened, y_train)
y_pred = rfr.predict(X_test_flattened)
classification_rep = classification_report(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
results["Random Forest"] = classification_rep, confusion_mat

Training Random Forest model.......


In [108]:
print("Classification Report for SVM:")
print(results["SVM"][0])
print("Confusion Matrix for SVM:")
print(results["SVM"][1])

Classification Report for SVM:
              precision    recall  f1-score   support

         0.0       0.69      1.00      0.81        22
         1.0       0.00      0.00      0.00        10

    accuracy                           0.69        32
   macro avg       0.34      0.50      0.41        32
weighted avg       0.47      0.69      0.56        32

Confusion Matrix for SVM:
[[22  0]
 [10  0]]


In [109]:
print("Classification Report for Random Forest:")
print(results["Random Forest"][0])
print("Confusion Matrix for Random Forest:")
print(results["Random Forest"][1])

Classification Report for Random Forest:
              precision    recall  f1-score   support

         0.0       0.73      0.86      0.79        22
         1.0       0.50      0.30      0.38        10

    accuracy                           0.69        32
   macro avg       0.62      0.58      0.58        32
weighted avg       0.66      0.69      0.66        32

Confusion Matrix for Random Forest:
[[19  3]
 [ 7  3]]


In [110]:
with open(r"Flask_webapp\models\rfr.pkl", "wb") as f:
    pickle.dump(rfr, f)

In [111]:
with open(r"Flask_webapp\models\svc.pkl", "wb") as f:
    pickle.dump(svc, f)

In [115]:
import cv2
import numpy as np

rfr = pickle.load(open(r"Flask_webapp\models\rfr.pkl", "rb"))
image_path = r"WhatsApp Image 2024-04-13 at 08.59.04_11efa25d.jpg"
image = cv2.imread(image_path)
image = cv2.resize(image, (224, 224))
image = image / 255.0
image_flat = image.flatten().reshape(1, -1)

predicted_class = rfr.predict(image_flat)
if predicted_class == 0:
    print("Deep Fake")
else:
    print("Real")

predicted_class[0]

Deep Fake


0.0